In [1]:
import os
os.chdir("..")

In [2]:
%pwd

'd:\\Portfolio Projects\\End-to-End-Medical-Chatbot-Generative-AI'

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# Extract Data From the PDF File
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf_file(data = 'data/')

In [6]:
# extracted_data

In [7]:
# Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("Length of the Text Chunks : ", len(text_chunks))

Length of the Text Chunks :  5860


In [9]:
# text_chunks

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
# Download the Embeddings from Hugging Face
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2') # 384 dimensional
    return embeddings

In [12]:
embeddings = download_hugging_face_embedding()

C:\Users\vrani\AppData\Local\Temp\ipykernel_6164\2655220996.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2') # 384 dimensional
c:\Users\vrani\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [13]:
# from sentence_transformers import SentenceTransformer

In [14]:
query_result = embeddings.embed_query("Hello World")
print("Length : ", len(query_result))

Length :  384


In [15]:
# query_result

In [42]:
from dotenv import load_dotenv
load_dotenv()

True

In [43]:
PINECONE_API_KEY = os.environ.get("MediBot_API_KEY")
COHERE_API_KEY = os.environ.get("COHERE_API_KEY")

In [23]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

c:\Users\vrani\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.30.2 at db_data_2025-01.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


{
    "name": "medibot",
    "metric": "cosine",
    "host": "medibot-dc1h4ph.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [44]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['COHERE_API_KEY'] = COHERE_API_KEY

In [24]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [25]:
# Load Existing Index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [26]:
docsearch

In [27]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs = {"k": 3})

In [28]:
retrieved_docs = retriever.invoke("What is Acne?")

In [29]:
retrieved_docs

[Document(id='0af4be3c-c72b-48a8-b2b9-91c3fbfaf9d1', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_Book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='0ef9011e-1fdc-47c9-8b2f-95a915fefc4f', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_Book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [48]:
from langchain.llms import Cohere

# Set up Cohere LLM with your API key
llm = Cohere()

In [49]:
from langchain.chains import create_retrieval_chain 
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the questions. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the" \
    "answer concise." \
    "\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)

In [50]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [52]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

 Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. It is characterized by blackheads, whiteheads, and inflamed pimples, and sometimes cysts. 

There are three main types of acne:
1. Acne Vulgaris: This is the most common form of acne and is characterized by the occurrence of pimples, blackheads, and whiteheads on the face, neck, chest, and back. 

2. Acne Conglobata: This is a more severe and uncommon form of acne that occurs when smaller areas of acne come together to form large lumps. It often occurs on the back, chest, and shoulders.

3. Acne Rosacea: This type of acne occurs more frequently in adults and is characterized by prominent blood vessels, swelling, and redness on the face. 

I hope this helps, let me know if you'd like more information! 


In [53]:
response = rag_chain.invoke({"input": "What is Stats?"})
print(response["answer"])

 I don't know the answer to that. Please define your question, and provide some context of what you want to know. 

I can provide information about statistical topics, numerical data, and computational techniques if you'd like me to. 
